In [78]:
import pandas as pd
import numpy as np
df = pd.read_csv(
    "../backend/logging/search.log", 
    sep=";", 
    header=None,  
    names=['time', 'event', 'query', 'documents']  # Column names
)

In [79]:
groups = df.groupby(by=["query"])

with open("pre_file.csv", "w") as f2:

    for name, data in groups:
        vector_lines = list(data.to_numpy())

        time, event, query, documents = vector_lines.pop(0)

        clicks = []

        for time_local, event_local, query_local, documents_local in vector_lines:
            
            if event_local == "Search.clicked":

                clicks.append(documents_local)

            else:
                
                if len(clicks) != 0:
                    documents_local_rank = documents.split("|")
                    last = clicks[-1]

                    position = documents_local_rank.index(last)
                    put_file_documents_local_rank = documents_local_rank[:position+2]

                    
                    for doc in put_file_documents_local_rank:

                        f2.write(f"{query_local},{doc},1,0\n")
                    
                    for click in clicks:

                        f2.write(f"{query_local},{click},0,1\n")

                time, event, query, documents = time_local, event_local, query_local, documents_local
                
                clicks= []

        if len(clicks) != 0:
            documents_local_rank = documents.split("|")
            last = clicks[-1]

            position = documents_local_rank.index(last)
            put_file_documents_local_rank = documents_local_rank[:position+2]

     
            
            for doc in put_file_documents_local_rank:

                f2.write(f"{query_local},{doc},1,0\n")
            
            for click in clicks:

                f2.write(f"{query_local},{click},0,1\n")

     

/tmp/ipykernel_24813/3529901427.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, data in groups:


In [80]:
!sort -t ',' -k 1n pre_file.csv > pre_file_sorted.csv

In [81]:
!rm pre_file.csv

In [82]:
df_pre_file = pd.read_csv("pre_file_sorted.csv", header=None, names=['query', 'document', 'view', 'click'])
group_pre_file = df_pre_file.groupby(by=['query','document'])

In [83]:
with open("features_file.csv", "w") as f4:
    
    for name, df_process in group_pre_file:

        views = df_process['view'].sum()
        clicks = df_process['click'].sum()

        f4.write(f"{name[0]},{df_process['document'].unique()[0]},{views},{clicks}\n")
    

In [84]:
import numpy as np
with open("features_file.csv", "r") as f5, open("reference_colletion.csv", "w") as f6:

    f6.write(f"query,document,label\n")

    for line_ in f5.readlines():

        list_all = [x.strip() for x in line_.split(",")]

        click, view, document = list_all.pop(),list_all.pop(),list_all.pop()
        query = " ".join(list_all)

        view, click = int(view), int(click)
        prior_a = 5
        prior_b = 25
        posterior_a = click+prior_a
        posterior_b = prior_b+(view-click)
        label = np.round(((posterior_a  / (posterior_b+posterior_a))*32)).astype("int")

        if label > 31:
            print(document)
            raise ValueError(label)
        f6.write(f"{query},{document},{label}\n")